In [2]:
pip install pyathena

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Libraries
from pyathena import connect
import pandas as pd
import datetime as dt
from datetime import datetime

In [4]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'daily_check'

In [5]:
#Connection to daily_check (based on receiver)
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')
df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)

/tmp/ipykernel_4360/2235117015.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101970 entries, 0 to 101969
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   payer         101970 non-null  object 
 1   country       101970 non-null  object 
 2   date          101970 non-null  object 
 3   tx            101970 non-null  int64  
 4   amount        101970 non-null  float64
 5   coupon_count  101970 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 4.7+ MB


In [7]:
#Connection to daily_forex 
forex_table = 'last_daily_forex'
database_name= 'viamericas'
df_rates = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{forex_table};', conn)

/tmp/ipykernel_4360/3034134324.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_rates = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{forex_table};', conn)


In [8]:
# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Grouping by 'payer' and 'country' concatenated for this level of granularity
df['payer_country'] = df['payer'] + '_' + df['country']

In [9]:
df = df[df['date'] != '2020-12-31'] # Excluyo el 31-12-2020
df = df[df['date'] <= '2023-10-21'] # Cortamos el dataset hasta el 21/10/2023

In [10]:
df.date.max()

Timestamp('2023-10-21 00:00:00')

In [11]:
def aging_filter(df):
    # Find the last date in the sample
    last_date_sample = df['date'].max()

    # Calculate the limit date, one day before the last date in the sample
    limit_date = last_date_sample - pd.Timedelta(days=1)

    # Find the first and last date per 'payer_country' and calculate total_amount and count
    result = (
        df.groupby('payer_country')
        .agg(
            first_date=('date', 'min'),
            last_date=('date', 'max'),
            total_amount=('amount', 'sum'),
            total_transactions=('tx', 'sum')  # Add this line for total_transactions
        )
        .reset_index()
    )

    # Calculate the 'age_payer' variable accurately using the calculated limit_date
    result['age_payer'] = ((limit_date - result['first_date']).dt.days / 30).round(2)

    # Calculate the 'active_payer' variable
    result['active_time'] = ((result['last_date'] - result['first_date']).dt.days / 30).round(2)

    # Calculate the 'inactive' variable
    result['inactive_time'] = ((limit_date - result['last_date']).dt.days / 30).round(2)

    # Sort the final DataFrame by 'total_amount' from highest to lowest
    result = result.sort_values(by='total_amount', ascending=False)

    # Filter the DataFrame based on conditions and return aging_universe
    aging_universe = result.loc[
        (result.age_payer >= 3) & 
        (result.inactive_time <= 3) & 
        (result.total_amount > 10000) & 
        (result.total_transactions > 50)
    ]

    return aging_universe

In [12]:
df['country'].unique()

array(['EL SALVADOR', 'ECUADOR', 'BOLIVIA', 'HONDURAS', 'MEXICO',
       'UNITED STATES', 'PHILIPPINES', 'HAITI', 'GUATEMALA', 'COLOMBIA',
       'NIGERIA', 'UNITED STATES (ATM & DEPOSITS)', 'COSTA RICA', 'PERU',
       'NICARAGUA', 'BRAZIL', 'DOMINICAN REPUBLIC', 'MALI', 'LIBERIA',
       'INDIA', 'ARGENTINA', 'BENIN', 'BURKINA FASO', 'SENEGAL', 'GHANA',
       'SIERRA LEONE', 'CHILE', 'SPAIN', 'BANGLADESH', 'ITALY', 'TOGO',
       'INDONESIA', 'NEPAL', 'GUINEA', 'GAMBIA, THE', 'GERMANY',
       'THAILAND', 'MALAYSIA', 'UNITED KINGDOM', 'BELGIUM',
       "COTE D'IVOIRE (IVORY COAST)", 'CAMBODIA', 'JAMAICA', 'CAMEROON',
       'URUGUAY', 'PORTUGAL', 'RWANDA', 'FRANCE', 'NETHERLANDS',
       'SWITZERLAND', 'FIJI', 'UGANDA', 'ZAMBIA', 'KENYA', 'TANZANIA',
       'CZECH REPUBLIC', 'ROMANIA', 'SRI LANKA', 'LITHUANIA', 'ETHIOPIA',
       'CONGO, (CONGO Â– BRAZZAVILLE)', 'MAURITANIA', 'IRELAND',
       'AUSTRIA', 'QATAR', 'GABON', 'VIETNAM', 'NORWAY', 'PAKISTAN',
       'LUXEMBOURG', 'CHAD',

In [13]:
# Defining Universe
df = df[df['amount'] != 0] # Excluding 0 (flag A & Flag C), defined in EDA - CHEQUEAR!

df_aging = aging_filter(df) #Filtering 'payer_country' based on Aging notebook
df_filtered = df[df['payer_country'].isin(df_aging['payer_country'])] # Applying aging filters 
df_filtered['date'] = pd.to_datetime(df_filtered['date']).dt.date

/tmp/ipykernel_4360/3736126815.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = pd.to_datetime(df_filtered['date']).dt.date


In [14]:
# FOREX - Selecting columns & renaming
df_rates=df_rates.rename(columns={'day': 'date', 'max_feed_price': 'feed_price'})
df_rates=df_rates.loc[:,['date', 'feed_price', 'symbol']]

In [15]:
df_rates

,date,feed_price,symbol
0,2023-05-28,4.9947,USDBRL
1,2023-05-28,82.6542,USDINR
2,2023-05-28,7.8211,USDGTQ
3,2023-05-28,17.6357,USDMXN
4,2023-05-28,55.9804,USDPHP
...,...,...,...
5110,2022-01-14,74.3865,USDINR
5111,2022-01-14,51.4457,USDPHP
5112,2022-01-14,5.5400,USDBRL
5113,2022-01-14,20.3487,USDMXN


In [16]:
def get_closing_prices(data, start_date, end_date):
    # Convert the 'FEED_DATE' column to datetime
    data['date'] = pd.to_datetime(data['date'])

    # Filter by date range
    data_filtered = data[(data['date'] >= start_date) & (data['date'] <= end_date)]

    # Sort by 'FEED_DATE' > chronological order
    data_filtered = data_filtered.sort_values(by='date')

    # Group by 'SYMBOL' and 'FEED_DATE', taking the last value of 'FEED_PRICE'
    data_grouped = data_filtered.groupby(['symbol', pd.Grouper(key='date', freq='D')])['feed_price'].last().reset_index()
    
    return data_grouped

In [17]:
start_date = '2021-01-01'
end_date = '2023-10-21'

rates = get_closing_prices(df_rates, start_date, end_date)

In [18]:
###WE OBTAIN THE LAGS FOR THE EXCHANGE RATE 

def generate_lag_and_variation(df, num_lags):
    # Create columns for each day's lag up to the defined maximum
    for i in range(1, num_lags + 1):
        col_name = f'rate_lag_{i}'
        # Shift the 'feed_price' column grouped by 'symbol'
        df[col_name] = df.groupby('symbol')['feed_price'].shift(i)

    # Calculate the variation columns between consecutive delays
    for i in range(1, num_lags):
        col_name = f'var_rate_lag_{i}'
        # Calculate the difference between consecutive delay columns
        df[col_name] = df[f'rate_lag_{i}'] - df[f'rate_lag_{i + 1}']
    
    return df

In [19]:
rates_number = 30
rates = generate_lag_and_variation(rates, rates_number)
rates['date'] = rates['date'].dt.date #Date format

In [20]:
rates

,symbol,date,feed_price,rate_lag_1,rate_lag_2,rate_lag_3,rate_lag_4,rate_lag_5,rate_lag_6,rate_lag_7,...,var_rate_lag_20,var_rate_lag_21,var_rate_lag_22,var_rate_lag_23,var_rate_lag_24,var_rate_lag_25,var_rate_lag_26,var_rate_lag_27,var_rate_lag_28,var_rate_lag_29
0,USDBRL,2021-01-01,5.1935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USDBRL,2021-01-02,5.1934,5.1935,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USDBRL,2021-01-03,5.2008,5.1934,5.1935,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USDBRL,2021-01-04,5.2976,5.2008,5.1934,5.1935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USDBRL,2021-01-05,5.3244,5.2976,5.2008,5.1934,5.1935,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5110,USDPHP,2023-10-16,56.8185,56.8485,56.8095,56.8227,56.8275,56.8185,56.9342,56.9450,...,0.3564,0.0656,0.0000,0.0000,-0.0257,-0.0093,0.0891,-0.0782,-0.0033,0.1549
5111,USDPHP,2023-10-17,56.7100,56.8185,56.8485,56.8095,56.8227,56.8275,56.8185,56.9342,...,-0.0893,0.3564,0.0656,0.0000,0.0000,-0.0257,-0.0093,0.0891,-0.0782,-0.0033
5112,USDPHP,2023-10-18,56.8872,56.7100,56.8185,56.8485,56.8095,56.8227,56.8275,56.8185,...,-0.1337,-0.0893,0.3564,0.0656,0.0000,0.0000,-0.0257,-0.0093,0.0891,-0.0782
5113,USDPHP,2023-10-19,56.9022,56.8872,56.7100,56.8185,56.8485,56.8095,56.8227,56.8275,...,-0.3476,-0.1337,-0.0893,0.3564,0.0656,0.0000,0.0000,-0.0257,-0.0093,0.0891


In [21]:
rates_dict = {
    'USDBRL': 'BRAZIL', # Bz Real 
    'USDINR': 'INDIA', # Indian Rupia
    'USDGTQ': 'GUATEMALA', #Quetzal 
    'USDMXN': 'MEXICO', #Mx Peso
    'USDPHP': 'PHILIPPINES' # Ph Peso
}

In [22]:
rates['country'] = rates['symbol'].map(rates_dict)

In [23]:
rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5115 entries, 0 to 5114
Data columns (total 63 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   symbol           5115 non-null   object 
 1   date             5115 non-null   object 
 2   feed_price       5115 non-null   float64
 3   rate_lag_1       5110 non-null   float64
 4   rate_lag_2       5105 non-null   float64
 5   rate_lag_3       5100 non-null   float64
 6   rate_lag_4       5095 non-null   float64
 7   rate_lag_5       5090 non-null   float64
 8   rate_lag_6       5085 non-null   float64
 9   rate_lag_7       5080 non-null   float64
 10  rate_lag_8       5075 non-null   float64
 11  rate_lag_9       5070 non-null   float64
 12  rate_lag_10      5065 non-null   float64
 13  rate_lag_11      5060 non-null   float64
 14  rate_lag_12      5055 non-null   float64
 15  rate_lag_13      5050 non-null   float64
 16  rate_lag_14      5045 non-null   float64
 17  rate_lag_15   

In [24]:
df1 = pd.merge(df_filtered, rates, on=['date', 'country'], how='left')
df1['date'] = pd.to_datetime(df1['date']).dt.date

In [25]:
### EFFECT OF CANCELED TRANSACTIONS ###
database_name= 'analytics'
table2_name = 'daily_sales_count_cancelled_v2'##WE LOAD THE BASE WITH CANCELLATIONS

df2 = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table2_name};', conn)
df2['date'] = pd.to_datetime(df2['date'])
df2 = df2[df2['date'] != '2020-12-31'] # Elimino el registro del 31.12.2020

/tmp/ipykernel_4360/1901381988.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table2_name};', conn)


In [26]:
from sklearn.preprocessing import LabelEncoder

# Grouping by 'payer' and 'country' concatenated for this level of granularity
df2['payer_country'] = df2['payer'] + '_' + df2['country']
# Inicializing LabelEncoder
label_encoder = LabelEncoder()
# Coding ‘PAYER_COUNTRY’ as unique values
df2['payer_country_encoder'] = label_encoder.fit_transform(df2['payer_country'])

In [27]:
# By applying the same aging filter, we can work on the same payer_country universe 
df2 = df2[df2['payer_country'].isin(df_aging['payer_country'])]

In [28]:
def fill_missing_dates(df, start_date, end_date):
    # Create an empty DataFrame with the specified date range
    date_range = pd.date_range(start=start_date, end=end_date)
    df_fill = pd.DataFrame({'date': date_range, 'amount': 0, 'tx_cancelled': 0})
    df_fill['date'] = pd.to_datetime(df_fill['date']).dt.date
    
    # Sort the original DataFrame by 'country', 'payer', and 'date'
    df = df.sort_values(by=['country', 'payer', 'date'])
    
    # Create an empty DataFrame to hold the result
    result_df = pd.DataFrame()

    # Loop through each 'payer_country'
    for payer_country in df['payer_country'].unique():
#        print(payer_country)
        df_aux = df[df['payer_country'] == payer_country]
        # Combining df_aux (payer_country) with df_fill, keeping values from df_aux and filling dates, amount and tx_cancelled 
        merged_df = df_aux.set_index('date').combine_first(df_fill.set_index('date')).reset_index()

        # Fill missing values in 'payer', 'country', 'payer_country', 'payer_country_encoder' columns
        columns_to_fill = ['payer', 'country', 'payer_country', 'payer_country_encoder']
        merged_df[columns_to_fill] = merged_df[columns_to_fill].ffill().bfill()

        result_df = pd.concat([result_df, merged_df], ignore_index=True)

    
    return result_df

In [29]:
#start_date and end_date variables defined previously
start_date = '2021-01-01'
end_date = '2023-10-21'

# Call the function with the specified start_date and end_date
df_full = fill_missing_dates(df2, start_date, end_date)

In [30]:
def generate_tx_lags_and_variation(df, tx_count):
    """
    Generate lag columns for cancelled transactions and their variations.

    Args:
    - df: DataFrame containing transaction data
    - tx_count: Number of periods for lag calculation

    Returns:
    - df: DataFrame with added lag and variation columns
    """
    # Sort the dataset based on country, payer, and date
    df = df.sort_values(by=['country', 'payer', 'date'])

    # Create columns for each day's lag up to the defined maximum
    for i in range(1, tx_count + 1):
        col_name = f'tx_cancelled_lag_{i}'
        # Shift the 'tx_cancelled' column grouped by 'country' and 'payer'
        df[col_name] = df.groupby(['country', 'payer'])['tx_cancelled'].shift(i)

    # Calculate the variation columns between consecutive delays
    for i in range(1, tx_count):
        col_name = f'var_tx_cancelled_lag_{i}'
        # Calculate the difference between consecutive lag columns
        df[col_name] = df[f'tx_cancelled_lag_{i}'] - df[f'tx_cancelled_lag_{i + 1}']

    return df


In [31]:
# Call the function and assign the result back to df2
tx_cancelled_lags = 30
df2 = generate_tx_lags_and_variation(df_full, tx_cancelled_lags)

In [32]:
df2['date'] = df2['date'].dt.date

In [33]:
df1['ratio_coupon_tx']=df1.coupon_count/df1.tx

In [34]:
df1.ratio_coupon_tx.min();df1.ratio_coupon_tx.max()

5.0

In [35]:

def generate_coupon_tx_lags(df, tx_count):
    """
    Generate lag columns for coupon_tx ratio

    Args:
    - df: DataFrame containing transaction data
    - tx_count: Number of periods for lag calculation

    Returns:
    - df: DataFrame with added lag and variation columns
    """
    # Sort the dataset based on country, payer, and date
    df = df.sort_values(by=['country', 'payer', 'date'])

    # Create columns for each day's lag up to the defined maximum
    for i in range(1, tx_count + 1):
        col_name = f'ratio_coupon_tx_lag_{i}'
        # Shift the 'ratio_coupon_tx' column grouped by 'country' and 'payer'
        df[col_name] = df.groupby(['country', 'payer'])['ratio_coupon_tx'].shift(i)

    return df


In [36]:
# Call the function and assign the result back to df1
tx_ratio_coupon_tx_lags = 30
df1 = generate_coupon_tx_lags(df1, tx_ratio_coupon_tx_lags)

In [37]:
def generate_tx_lags(df, tx_count):
    """
    Generate lags columns for txs

    Args:
    - df: DataFrame containing transaction data
    - tx_count: Number of periods for lag calculation

    Returns:
    - df: DataFrame with added lag and variation columns
    """
    # Sort the dataset based on country, payer, and date
    df = df.sort_values(by=['country', 'payer', 'date'])

    # Create columns for each day's lag up to the defined maximum
    for i in range(1, tx_count + 1):
        col_name = f'tx_lag_{i}'
        # Shift the 'tx' column grouped by 'country' and 'payer'
        df[col_name] = df.groupby(['country', 'payer'])['tx'].shift(i)

    return df

In [38]:
# Call the function and assign the result back to df1
tx_lags = 30
df1 = generate_tx_lags(df1, tx_lags)

In [39]:
df_final = pd.merge(df1,df2, on=['date','payer','country', 'amount'], how='outer')
df_final['date'] = pd.to_datetime(df_final['date'])

In [40]:
#for x in df_final.columns:
#   print (x)

In [41]:
###WE SHOULD KEEP THE VARIABLES THAT HAVE IMPUTED MISSINGS IN AMOUNT AND TX_CANCELLED (DAYS WITHOUT OPERATIONS) BUT KEEP THE MISSINGS IN RATES
df_final=df_final.loc[:,['date', 'payer', 'country', 'amount', 
                         'var_rate_lag_1', 'var_rate_lag_2', 'var_rate_lag_3',
                         'var_rate_lag_4', 'var_rate_lag_5', 'var_rate_lag_6', 'var_rate_lag_7',
                         'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10',
                         'var_rate_lag_11', 'var_rate_lag_12', 'var_rate_lag_13',
                         'var_rate_lag_14', 'var_rate_lag_15', 'var_rate_lag_16',
                         'var_rate_lag_17', 'var_rate_lag_18', 'var_rate_lag_19',
                         'var_rate_lag_20', 'var_rate_lag_21', 'var_rate_lag_22',
                         'var_rate_lag_23', 'var_rate_lag_24', 'var_rate_lag_25',
                         'var_rate_lag_26', 'var_rate_lag_27', 'var_rate_lag_28',
                         'var_rate_lag_29', 
                         'payer_country_encoder','payer_country_y',
                         'var_tx_cancelled_lag_1', 'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3',
                         'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6',
                         'var_tx_cancelled_lag_7', 'var_tx_cancelled_lag_8', 'var_tx_cancelled_lag_9',
                         'var_tx_cancelled_lag_10', 'var_tx_cancelled_lag_11', 'var_tx_cancelled_lag_12',
                         'var_tx_cancelled_lag_13', 'var_tx_cancelled_lag_14', 'var_tx_cancelled_lag_15',
                         'var_tx_cancelled_lag_16', 'var_tx_cancelled_lag_17', 'var_tx_cancelled_lag_18',
                         'var_tx_cancelled_lag_19', 'var_tx_cancelled_lag_20', 'var_tx_cancelled_lag_21',
                         'var_tx_cancelled_lag_22', 'var_tx_cancelled_lag_23', 'var_tx_cancelled_lag_24',
                         'var_tx_cancelled_lag_25', 'var_tx_cancelled_lag_26', 'var_tx_cancelled_lag_27',
                         'var_tx_cancelled_lag_28', 'var_tx_cancelled_lag_29', 'var_tx_cancelled_lag_6',
                         'ratio_coupon_tx_lag_1', 'ratio_coupon_tx_lag_2', 'ratio_coupon_tx_lag_3',
                         'ratio_coupon_tx_lag_4', 'ratio_coupon_tx_lag_5', 'ratio_coupon_tx_lag_6',
                          'ratio_coupon_tx_lag_7', 'ratio_coupon_tx_lag_8', 'ratio_coupon_tx_lag_9',
                          'ratio_coupon_tx_lag_10', 'ratio_coupon_tx_lag_11', 'ratio_coupon_tx_lag_12',
                          'ratio_coupon_tx_lag_13', 'ratio_coupon_tx_lag_14', 'ratio_coupon_tx_lag_15',
                          'ratio_coupon_tx_lag_16', 'ratio_coupon_tx_lag_17', 'ratio_coupon_tx_lag_18',
                          'ratio_coupon_tx_lag_19', 'ratio_coupon_tx_lag_20', 'ratio_coupon_tx_lag_21',
                          'ratio_coupon_tx_lag_22', 'ratio_coupon_tx_lag_23', 'ratio_coupon_tx_lag_24',
                          'ratio_coupon_tx_lag_25', 'ratio_coupon_tx_lag_26', 'ratio_coupon_tx_lag_27',
                          'ratio_coupon_tx_lag_28', 'ratio_coupon_tx_lag_29',
                          'tx_lag_1', 'tx_lag_2','tx_lag_3','tx_lag_4','tx_lag_5','tx_lag_6','tx_lag_7',
                          'tx_lag_8', 'tx_lag_9','tx_lag_10','tx_lag_11','tx_lag_12','tx_lag_13','tx_lag_14',
                          'tx_lag_15', 'tx_lag_16','tx_lag_17','tx_lag_18','tx_lag_19','tx_lag_20','tx_lag_21',
                          'tx_lag_22', 'tx_lag_23','tx_lag_24','tx_lag_25','tx_lag_26','tx_lag_27','tx_lag_28',
                          'tx_lag_29',]]
                        
        
df_final = df_final.rename(columns={'payer_country_y': 'payer_country'})

In [42]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133120 entries, 0 to 133119
Columns: 123 entries, date to tx_lag_29
dtypes: datetime64[ns](1), float64(119), object(3)
memory usage: 124.9+ MB


In [43]:
### DUMMIES ####

# Weekend indicator (dummy)
df_final['weekend'] = (df_final['date'].dt.dayofweek >= 4).astype(int)

In [44]:
#WE ADD ONE MORE DAY AFTER CHRISTMAS, NEW YEAR'S EVE AND US INDEPENDENCE (BETTER CAPTURES THE DISTURBANCE).
# Special dates list - Format: Month-day 
special_dates_list = [ '07-04', #US independence day 
                      '12-25', #Christmas
                      '05-11' # Mexico & Colombia - Mother's day
                     '01-01', #New Year
                     '01-02',
                     '12-26',
                     '07-05'] 

# Applying format process special_dates function
special_dates_list_processed = [(int(date.split('-')[0]), int(date.split('-')[1])) for date in special_dates_list]

In [45]:
# To check special dates defined by list
def mark_special_dates(row):
    if (row['date'].month, row['date'].day) in special_dates_list_processed:
        return 1
    else:
        return 0

In [46]:
# Marking special dates in df
df_final['special_dates'] = df_final.apply(lambda row: mark_special_dates(row), axis=1)

In [47]:
df_final['amount'].sum()

24303285928.466

In [48]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133120 entries, 0 to 133119
Columns: 125 entries, date to special_dates
dtypes: datetime64[ns](1), float64(119), int64(2), object(3)
memory usage: 127.0+ MB


In [49]:
# Impute missings in exogenous variables
df_final.fillna(0, inplace=True)

In [50]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133120 entries, 0 to 133119
Columns: 125 entries, date to special_dates
dtypes: datetime64[ns](1), float64(119), int64(2), object(3)
memory usage: 127.0+ MB


In [51]:
##mantenemos mismos id_payer que antes##
del df_final['payer_country_encoder']
label_encoder = LabelEncoder()
# Coding ‘PAYER_COUNTRY’ as unique values
df_final['payer_country_encoder'] = label_encoder.fit_transform(df_final['payer_country'])

In [52]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133120 entries, 0 to 133119
Columns: 125 entries, date to payer_country_encoder
dtypes: datetime64[ns](1), float64(118), int64(3), object(3)
memory usage: 127.0+ MB


In [53]:
df_final['amount'].sum()

24303285928.466

In [54]:
df_final.date.max(),df_final.date.min()

(Timestamp('2023-10-21 00:00:00'), Timestamp('2021-01-01 00:00:00'))

In [55]:
df_final.dtypes

date                     datetime64[ns]
payer                            object
country                          object
amount                          float64
var_rate_lag_1                  float64
                              ...      
tx_lag_28                       float64
tx_lag_29                       float64
weekend                           int64
special_dates                     int64
payer_country_encoder             int64
Length: 125, dtype: object

In [348]:
##Top 10 payers##
#total_amount_per_payer = df_final.groupby('payer_country_encoder')['amount'].sum()
#top_10_payers = total_amount_per_payer.sort_values(ascending=False).head(10)
#print(top_10_payers)

In [349]:
#top_10_payers=[57,29,36,24,38,60,32,20,19,101]
#top_10 = df_final[df_final['payer_country_encoder'].isin(top_10_payers)]

In [350]:
#top_10.country.unique()

In [351]:
#top_10.loc[top_10.country=='HONDURAS']

In [352]:
control=df_final.loc[df_final.payer_country_encoder==57]

In [353]:
#control.tail()

In [354]:
#control.amount.sum()
#df_final.amount.sum()

In [355]:
!pip install awswrangler

  Using cached awswrangler-3.5.2-py3-none-any.whl.metadata (23 kB)
Using cached awswrangler-3.5.2-py3-none-any.whl (402 kB)


In [356]:
#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#
##ROUTINES FOR EXPORTING CSV TO VIAMERICAS S3##
#!pip install awswrangler
#import awswrangler as wr
#file_name_target = 'df_final_21_10_2023.csv'

#wr.s3.to_csv(
    df=df_final,
    path=f's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/prueba_sk_forecast/{file_name_target}',
    dataset=False,
    index=False
)

{'paths': ['s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/prueba_sk_forecast/df_final_21_10_2023.csv'],
 'partitions_values': {}}